# Topic 7: Persistence and Checkpointing

Learn how to build stateful applications that remember conversations, survive restarts, and resume interrupted workflows.

## What is Persistence?

Persistence in LangGraph allows your graphs to:
- **Remember conversations** across multiple interactions
- **Resume workflows** after interruptions or crashes
- **Time travel** through previous states for debugging
- **Enable human-in-the-loop** patterns with approval workflows

### Why Does This Matter?

Imagine building:
- A chatbot that remembers what you discussed yesterday
- A multi-step workflow that can pause and resume
- An agent that waits for human approval before proceeding
- A system that can rollback to previous states if something goes wrong

All of this requires persistence!

## Learning Objectives

- Understand checkpoint-based persistence
- Use MemorySaver for in-memory state storage
- Build stateful chatbots with conversation memory
- Resume interrupted workflows
- Implement human-in-the-loop patterns
- Navigate through execution history

In [ ]:
# Setup
import os
import getpass
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

model = ChatAnthropic(model="claude-sonnet-4-20250514")
print("✓ Setup complete!")

## Understanding Checkpoints

A **checkpoint** is a snapshot of your graph's state at a specific point in time.

### How Checkpointing Works

```
Execution Flow:
[Start] → [Node 1] → [Checkpoint 1] → [Node 2] → [Checkpoint 2] → [End]
           ↓                           ↓
         Saved                      Saved
```

Each checkpoint saves:
- Current state values
- Which node just executed
- What node comes next
- Complete execution history

### MemorySaver: Your First Checkpointer

MemorySaver is LangGraph's built-in checkpointer that stores state in memory. It's perfect for:
- Development and testing
- Single-session applications
- Learning how persistence works

**Note**: MemorySaver data is lost when your program stops. For production, you'd use SqliteSaver or a custom database-backed checkpointer.

## Example 1: Stateful Chatbot with Memory

Let's build a chatbot that remembers your conversation across multiple interactions.

In [ ]:
# Define state for our chatbot
class ChatState(TypedDict):
    messages: Annotated[list, add_messages]

# Create a simple chatbot node
def chatbot_node(state: ChatState):
    """Process messages and generate a response."""
    response = model.invoke(state["messages"])
    return {"messages": [response]}

print("✓ Chatbot node created!")

In [ ]:
# Build the graph WITH persistence
graph_builder = StateGraph(ChatState)

# Add the chatbot node
graph_builder.add_node("chatbot", chatbot_node)

# Add edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# ⭐ The magic happens here: add a checkpointer!
memory = MemorySaver()
chatbot_graph = graph_builder.compile(checkpointer=memory)

print("✓ Chatbot graph compiled with persistence!")

### Using Thread IDs for Multiple Conversations

Each conversation needs a unique **thread_id** to keep conversations separate.

Think of thread_id as:
- A conversation ID
- A session identifier
- A user ID for multi-user systems

In [ ]:
# Configuration for this conversation thread
config = {"configurable": {"thread_id": "conversation-1"}}

# First message in our conversation
print("\n💬 Message 1:")
print("User: Hi! My name is Alice and I love Python programming.\n")

result = chatbot_graph.invoke(
    {"messages": [HumanMessage(content="Hi! My name is Alice and I love Python programming.")]},
    config=config
)

print(f"Assistant: {result['messages'][-1].content}")

In [ ]:
# Second message - same thread_id!
print("\n💬 Message 2:")
print("User: What's my name?\n")

result = chatbot_graph.invoke(
    {"messages": [HumanMessage(content="What's my name?")]},
    config=config  # Same thread_id as before!
)

print(f"Assistant: {result['messages'][-1].content}")
print("\n✓ The chatbot remembered your name from the previous message!")

In [ ]:
# Third message - testing deeper memory
print("\n💬 Message 3:")
print("User: What programming language did I mention?\n")

result = chatbot_graph.invoke(
    {"messages": [HumanMessage(content="What programming language did I mention?")]},
    config=config
)

print(f"Assistant: {result['messages'][-1].content}")
print("\n✓ Complete conversation history is maintained!")

### Multiple Concurrent Conversations

You can maintain separate conversations using different thread IDs:

In [ ]:
# Start a completely separate conversation
config_bob = {"configurable": {"thread_id": "conversation-2"}}

print("\n💬 Different conversation (thread-2):")
print("User: Hello! My name is Bob and I work with JavaScript.\n")

result = chatbot_graph.invoke(
    {"messages": [HumanMessage(content="Hello! My name is Bob and I work with JavaScript.")]},
    config=config_bob
)

print(f"Assistant: {result['messages'][-1].content}")

In [ ]:
# Ask the same question in Bob's conversation
print("\n💬 Same question, different thread:")
print("User: What's my name?\n")

result = chatbot_graph.invoke(
    {"messages": [HumanMessage(content="What's my name?")]},
    config=config_bob
)

print(f"Assistant: {result['messages'][-1].content}")
print("\n✓ This conversation knows about Bob, not Alice!")

In [ ]:
# Return to Alice's conversation
print("\n💬 Back to Alice's conversation (thread-1):")
print("User: Remind me what we talked about.\n")

result = chatbot_graph.invoke(
    {"messages": [HumanMessage(content="Remind me what we talked about.")]},
    config=config  # Back to thread-1
)

print(f"Assistant: {result['messages'][-1].content}")
print("\n✓ Alice's conversation history is intact!")

## Inspecting Conversation History

You can retrieve the complete state of any conversation using `get_state()`:

In [ ]:
# Get the current state of Alice's conversation
state = chatbot_graph.get_state(config)

print("\n📊 Conversation State for thread-1:")
print(f"\nNumber of messages: {len(state.values['messages'])}")
print(f"\nConversation history:")
print("=" * 60)

for i, msg in enumerate(state.values['messages'], 1):
    role = "User" if isinstance(msg, HumanMessage) else "Assistant"
    content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
    print(f"\n{i}. {role}: {content}")

## Example 2: Multi-Step Workflow with Interruption

Let's build a content approval workflow that can be interrupted and resumed:

```
Flow:
[Start] → [Generate Content] → [INTERRUPT] → [Approve/Reject] → [Publish] → [End]
                                    ↑
                              Wait for human
```

In [ ]:
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from typing import Literal

# Define workflow state
class WorkflowState(TypedDict):
    topic: str
    content: str
    approved: bool
    published: bool
    feedback: str

def generate_content(state: WorkflowState):
    """Generate initial content."""
    print(f"\n📝 Generating content about: {state['topic']}")
    
    prompt = f"Write a brief, engaging paragraph about {state['topic']}. Keep it under 100 words."
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {
        "content": response.content,
        "approved": False,
        "published": False,
        "feedback": ""
    }

def check_approval(state: WorkflowState) -> Literal["approved", "needs_revision"]:
    """Route based on approval status."""
    return "approved" if state.get("approved", False) else "needs_revision"

def publish_content(state: WorkflowState):
    """Publish the approved content."""
    print("\n✅ Publishing content...")
    return {"published": True}

def request_revision(state: WorkflowState):
    """Handle revision request."""
    print("\n🔄 Revision requested. Generating improved version...")
    
    prompt = f"""Improve this content based on the feedback:

ORIGINAL:
{state['content']}

FEEDBACK:
{state['feedback']}

Provide an improved version."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {
        "content": response.content,
        "approved": False,
        "feedback": ""
    }

print("✓ Workflow nodes defined!")

In [ ]:
# Build the workflow graph
workflow_builder = StateGraph(WorkflowState)

# Add nodes
workflow_builder.add_node("generate", generate_content)
workflow_builder.add_node("publish", publish_content)
workflow_builder.add_node("revise", request_revision)

# Add edges
workflow_builder.add_edge(START, "generate")
workflow_builder.add_conditional_edges(
    "generate",
    check_approval,
    {
        "approved": "publish",
        "needs_revision": END  # Pause here for human review
    }
)
workflow_builder.add_conditional_edges(
    "revise",
    check_approval,
    {
        "approved": "publish",
        "needs_revision": END  # Pause again if needed
    }
)
workflow_builder.add_edge("publish", END)

# Compile with checkpointer
workflow_memory = MemorySaver()
workflow_graph = workflow_builder.compile(checkpointer=workflow_memory)

print("✓ Workflow graph compiled with persistence!")

In [ ]:
# Visualize the workflow
from IPython.display import Image, display

try:
    display(Image(workflow_graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph structure:")
    print("START -> generate -> [approved?]")
    print("                      ├─ Yes -> publish -> END")
    print("                      └─ No -> END (wait for human)")

In [ ]:
# Start the workflow
workflow_config = {"configurable": {"thread_id": "workflow-1"}}

print("\n🚀 Starting content generation workflow...")
print("=" * 60)

result = workflow_graph.invoke(
    {
        "topic": "The benefits of LangGraph for building AI applications",
        "content": "",
        "approved": False,
        "published": False,
        "feedback": ""
    },
    config=workflow_config
)

print("\n📄 Generated Content:")
print("=" * 60)
print(result['content'])
print("\n⏸️  Workflow paused - waiting for approval...")

In [ ]:
# Get the current state to see where we are
current_state = workflow_graph.get_state(workflow_config)

print("\n📊 Current Workflow State:")
print("=" * 60)
print(f"Next node to execute: {current_state.next}")
print(f"Content generated: {bool(current_state.values['content'])}")
print(f"Approved: {current_state.values['approved']}")
print(f"Published: {current_state.values['published']}")

### Scenario 1: Approve the Content

Let's approve the content and resume the workflow:

In [ ]:
# Update the state to approve the content
print("\n✅ Approving content...")
print("=" * 60)

# Update state with approval
workflow_graph.update_state(
    workflow_config,
    {"approved": True}
)

# Resume workflow - pass None to continue from checkpoint
result = workflow_graph.invoke(None, config=workflow_config)

print("\n✓ Workflow resumed and completed!")
print(f"Published: {result['published']}")

### Scenario 2: Request Revision

Let's try the workflow again but request changes this time:

In [ ]:
# Start a new workflow
workflow_config_2 = {"configurable": {"thread_id": "workflow-2"}}

print("\n🚀 Starting new workflow...")
print("=" * 60)

result = workflow_graph.invoke(
    {
        "topic": "The importance of checkpointing in AI systems",
        "content": "",
        "approved": False,
        "published": False,
        "feedback": ""
    },
    config=workflow_config_2
)

print("\n📄 Generated Content:")
print("=" * 60)
print(result['content'])
print("\n⏸️  Workflow paused...")

In [ ]:
# Request revisions with feedback
print("\n🔄 Requesting revision...")
print("=" * 60)

workflow_graph.update_state(
    workflow_config_2,
    {
        "approved": False,
        "feedback": "Please add a specific example and make it more concise."
    },
    as_node="revise"  # Resume at the revise node
)

# Continue execution
result = workflow_graph.invoke(None, config=workflow_config_2)

print("\n📄 Revised Content:")
print("=" * 60)
print(result['content'])
print("\n⏸️  Workflow paused again for re-approval...")

In [ ]:
# Now approve and complete
print("\n✅ Approving revised content...")
print("=" * 60)

workflow_graph.update_state(
    workflow_config_2,
    {"approved": True}
)

result = workflow_graph.invoke(None, config=workflow_config_2)

print("\n✓ Workflow completed!")
print(f"Final state - Published: {result['published']}")

## Time Travel: Navigating Execution History

One powerful feature of checkpointing is the ability to view and restore previous states.

In [ ]:
# Get the complete history of a workflow
print("\n📜 Execution History for workflow-2:")
print("=" * 60)

history = list(workflow_graph.get_state_history(workflow_config_2))

print(f"\nTotal checkpoints: {len(history)}\n")

for i, state in enumerate(history[:5], 1):  # Show first 5 checkpoints
    print(f"{i}. Checkpoint ID: {state.config['configurable']['checkpoint_id'][:8]}...")
    print(f"   Next: {state.next}")
    print(f"   Approved: {state.values.get('approved', False)}")
    print(f"   Published: {state.values.get('published', False)}")
    print()

In [ ]:
# Replay from a specific checkpoint
if len(history) >= 2:
    print("\n⏮️  Demonstrating replay from earlier checkpoint...")
    print("=" * 60)
    
    # Get a checkpoint from history (the second one)
    earlier_checkpoint = history[1]
    
    print(f"\nReplaying from checkpoint: {earlier_checkpoint.config['configurable']['checkpoint_id'][:8]}...")
    print(f"State at that point:")
    print(f"  - Approved: {earlier_checkpoint.values.get('approved', False)}")
    print(f"  - Published: {earlier_checkpoint.values.get('published', False)}")
    print(f"  - Next node: {earlier_checkpoint.next}")
    print("\n✓ You can resume execution from any checkpoint!")

## Example 3: Research Task with Resumable Steps

Let's build a research workflow that can be interrupted and resumed at any step.

In [ ]:
class ResearchState(TypedDict):
    topic: str
    research_plan: str
    findings: str
    summary: str
    completed_steps: list[str]

def plan_research(state: ResearchState):
    """Create a research plan."""
    print("\n📋 Planning research...")
    
    prompt = f"Create a brief research plan for: {state['topic']}. List 3 key areas to investigate."
    response = model.invoke([HumanMessage(content=prompt)])
    
    completed = state.get('completed_steps', [])
    completed.append('plan')
    
    return {
        "research_plan": response.content,
        "completed_steps": completed
    }

def conduct_research(state: ResearchState):
    """Simulate conducting research."""
    print("\n🔬 Conducting research...")
    
    prompt = f"""Based on this research plan, provide key findings:

PLAN:
{state['research_plan']}

Provide 3-4 key findings."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    completed = state.get('completed_steps', [])
    completed.append('research')
    
    return {
        "findings": response.content,
        "completed_steps": completed
    }

def summarize_research(state: ResearchState):
    """Create final summary."""
    print("\n📊 Summarizing findings...")
    
    prompt = f"""Summarize these research findings in 2-3 sentences:

FINDINGS:
{state['findings']}"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    completed = state.get('completed_steps', [])
    completed.append('summarize')
    
    return {
        "summary": response.content,
        "completed_steps": completed
    }

print("✓ Research nodes defined!")

In [ ]:
# Build the research graph
research_builder = StateGraph(ResearchState)

research_builder.add_node("plan", plan_research)
research_builder.add_node("research", conduct_research)
research_builder.add_node("summarize", summarize_research)

research_builder.add_edge(START, "plan")
research_builder.add_edge("plan", "research")
research_builder.add_edge("research", "summarize")
research_builder.add_edge("summarize", END)

# Compile with persistence
research_memory = MemorySaver()
research_graph = research_builder.compile(checkpointer=research_memory)

print("✓ Research graph compiled!")

In [ ]:
# Start research (but we'll "interrupt" it)
research_config = {"configurable": {"thread_id": "research-1"}}

print("\n🚀 Starting research workflow...")
print("=" * 60)

# Run just the first step
for event in research_graph.stream(
    {
        "topic": "The impact of AI agents on software development",
        "research_plan": "",
        "findings": "",
        "summary": "",
        "completed_steps": []
    },
    config=research_config,
    stream_mode="values"
):
    if event.get('completed_steps'):
        print(f"\n✓ Completed steps: {event['completed_steps']}")
        
        # Simulate interruption after plan
        if 'plan' in event['completed_steps'] and len(event['completed_steps']) == 1:
            print("\n⚠️  Simulating interruption (e.g., server restart)...")
            print("=" * 60)
            break

In [ ]:
# Check the state after interruption
print("\n📊 Checking state after interruption...")
print("=" * 60)

interrupted_state = research_graph.get_state(research_config)

print(f"\nCompleted steps: {interrupted_state.values['completed_steps']}")
print(f"Next step: {interrupted_state.next}")
print(f"\nResearch plan exists: {bool(interrupted_state.values['research_plan'])}")
print(f"Findings exist: {bool(interrupted_state.values['findings'])}")
print(f"Summary exists: {bool(interrupted_state.values['summary'])}")

print("\n✓ State is persisted! We can resume from here.")

In [ ]:
# Resume the workflow from where it left off
print("\n🔄 Resuming workflow...")
print("=" * 60)

# Pass None to continue from checkpoint
final_result = research_graph.invoke(None, config=research_config)

print("\n✅ Research completed!")
print("=" * 60)
print(f"\nCompleted steps: {final_result['completed_steps']}")
print(f"\n📊 Final Summary:")
print(final_result['summary'])

## Best Practices for Persistence

### 1. Choose the Right Checkpointer

```python
# Development: MemorySaver (data lost on restart)
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

# Production: SqliteSaver (persists to disk)
from langgraph.checkpoint.sqlite import SqliteSaver
checkpointer = SqliteSaver.from_conn_string("checkpoints.db")

# Enterprise: PostgresSaver (scalable, multi-user)
from langgraph.checkpoint.postgres import PostgresSaver
checkpointer = PostgresSaver.from_conn_string("postgresql://...")
```

### 2. Use Meaningful Thread IDs

```python
# ❌ Bad: Generic IDs
config = {"configurable": {"thread_id": "1"}}

# ✅ Good: Descriptive IDs
config = {"configurable": {"thread_id": f"user-{user_id}-session-{session_id}"}}
```

### 3. Handle State Updates Carefully

```python
# ❌ Bad: Overwriting entire state
graph.update_state(config, new_state)  # Loses other fields

# ✅ Good: Partial updates
graph.update_state(config, {"approved": True})  # Only updates one field
```

### 4. Design for Interruption

```python
# ✅ Good: Track progress
class State(TypedDict):
    data: str
    completed_steps: list[str]  # Track what's done
    current_step: str            # Track where we are

def my_node(state):
    # Do work
    result = process_data(state['data'])
    
    # Update progress
    completed = state.get('completed_steps', [])
    completed.append('processing')
    
    return {
        "data": result,
        "completed_steps": completed
    }
```

### 5. Clean Up Old Checkpoints

```python
# Implement cleanup for long-running applications
import time

def cleanup_old_threads(checkpointer, max_age_days=7):
    """Remove checkpoints older than max_age_days."""
    # Implementation depends on your checkpointer type
    pass
```

### 6. Test Resume Scenarios

```python
# Always test that your graph can resume correctly
def test_resume():
    config = {"configurable": {"thread_id": "test-1"}}
    
    # Start workflow
    graph.invoke(initial_state, config=config)
    
    # Get state
    state = graph.get_state(config)
    assert state.next is not None
    
    # Resume
    result = graph.invoke(None, config=config)
    assert result['completed'] == True
```

### 7. Use Checkpoints for Debugging

```python
# View execution history to debug issues
history = list(graph.get_state_history(config))

for i, state in enumerate(history):
    print(f"Step {i}: {state.next}")
    print(f"Values: {state.values}")
    print()
```

## Common Patterns

### Pattern 1: Human-in-the-Loop Approval

```python
def needs_approval(state) -> Literal["approved", "pending"]:
    return "approved" if state.get("approved") else "pending"

# Add conditional that stops at approval
graph.add_conditional_edges(
    "generate",
    needs_approval,
    {
        "approved": "continue_workflow",
        "pending": END  # Pause here
    }
)

# Later, resume with approval
graph.update_state(config, {"approved": True})
graph.invoke(None, config=config)
```

### Pattern 2: Progress Tracking

```python
class ProgressState(TypedDict):
    steps_completed: int
    total_steps: int
    current_task: str

def show_progress(state):
    progress = (state['steps_completed'] / state['total_steps']) * 100
    print(f"Progress: {progress:.1f}% - {state['current_task']}")
```

### Pattern 3: Retry with Checkpoint

```python
def resilient_node(state):
    try:
        return process_data(state)
    except Exception as e:
        # State is automatically checkpointed
        # You can retry from the same point
        return {"error": str(e), "retry_count": state.get('retry_count', 0) + 1}

def should_retry(state) -> Literal["retry", "fail"]:
    return "retry" if state.get('retry_count', 0) < 3 else "fail"
```

## Exercise 1: Build a Multi-User Chat System

Create a chatbot that can handle multiple users simultaneously, each with their own conversation history.

Requirements:
1. Each user has a separate thread_id
2. Users can have ongoing conversations
3. Conversation history is maintained across interactions
4. Implement a way to view conversation history

In [ ]:
# Your code here!

# TODO: Define a ChatState with messages

# TODO: Create a chatbot node

# TODO: Build and compile the graph with MemorySaver

# TODO: Simulate conversations with 2-3 different users

# TODO: Show that each user's history is kept separate


## Exercise 2: Implement a Pausable Task Pipeline

Create a data processing pipeline that:
1. Reads data (simulated)
2. Processes data (simulated)
3. Validates results (simulated)
4. Saves results (simulated)

Requirements:
- Use checkpointing to track progress
- Add a manual approval step after validation
- Allow the workflow to be paused and resumed
- Track which steps have been completed

In [ ]:
# Your code here!

# TODO: Define a PipelineState with fields for each step

# TODO: Create nodes for read, process, validate, save

# TODO: Add a conditional edge that pauses after validation

# TODO: Build the graph with checkpointing

# TODO: Run the pipeline, pause it, and then resume


## Exercise 3: Checkpoint History Explorer

Create a utility that:
1. Executes a multi-step workflow
2. Retrieves the complete checkpoint history
3. Displays the state at each checkpoint
4. Allows "replaying" from a specific checkpoint

This will help you understand how checkpoint history works.

In [ ]:
# Your code here!

# TODO: Create a simple 3-step workflow

# TODO: Run it completely

# TODO: Use get_state_history() to retrieve all checkpoints

# TODO: Display information about each checkpoint

# TODO: Show how to replay from a specific checkpoint


## Key Takeaways

In this notebook, you learned:

1. ✅ What persistence and checkpointing are in LangGraph
2. ✅ How to use MemorySaver for in-memory state persistence
3. ✅ Building stateful chatbots that remember conversations
4. ✅ Creating workflows that can be interrupted and resumed
5. ✅ Using thread_ids to manage multiple concurrent conversations
6. ✅ Implementing human-in-the-loop approval patterns
7. ✅ Navigating through checkpoint history for debugging
8. ✅ Best practices for production persistence

## When to Use Persistence

Use checkpointing when you need:
- **Conversation memory** - Chatbots that remember context
- **Long-running workflows** - Tasks that take minutes or hours
- **Human approval** - Workflows that pause for human input
- **Fault tolerance** - Resume after crashes or restarts
- **Multi-session interactions** - Users returning to continue tasks
- **Debugging** - Inspect execution history and state evolution

## Production Considerations

For production applications:
1. Use **SqliteSaver** or **PostgresSaver** instead of MemorySaver
2. Implement **checkpoint cleanup** to manage storage
3. Add **error handling** for checkpoint operations
4. Consider **security** - who can access which threads?
5. Monitor **checkpoint size** - large states can impact performance
6. Implement **backup strategies** for critical workflows

## Next Steps

You've now mastered persistence in LangGraph! This is a crucial capability for building production-grade AI applications.

Continue exploring:
- **Database-backed persistence** with SqliteSaver
- **Streaming with checkpoints** for real-time updates
- **Complex multi-agent systems** with persistent state
- **Human-in-the-loop patterns** for AI safety

Congratulations on completing the Persistence module!